In [ ]:

import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp
from genpen.utils import Paper
from scipy import stats
import geopandas
from shapely.errors import TopologicalError
import functools
import vpype
from skimage import io
from pathlib import Path

from sklearn.preprocessing import minmax_scale
from skimage import feature
from skimage import exposure

from skimage import filters
from skimage.color import rgb2gray
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.morphology import disk
from numpy.random import default_rng

def local_angle(dx, dy):
    """Calculate the angles between horizontal and vertical operators."""
    return np.mod(np.arctan2(dy, dx), np.pi)

from PIL import Image


import cv2
from rasterio import features
import rasterio
%load_ext autoreload
%autoreload 2

In [ ]:
import pydiffvg as dg
import torch
import skimage
import numpy as np
from torchvision.transforms import functional as TF
from IPython import display

In [ ]:
def finalize_image(img, gamma = 2.2, normalize = False, as_Image=False):
    if not isinstance(img, np.ndarray):
        img = img.data.numpy()
    if normalize:
        img_rng = np.max(img) - np.min(img)
        if img_rng > 0:
            img = (img - np.min(img)) / img_rng
    img = np.clip(img, 0.0, 1.0)
    if img.ndim==2:
        #repeat along the third dimension
        img=np.expand_dims(img,2)
    img[:, :, :3] = np.power(img[:, :, :3], 1.0/gamma)
    img = (img * 255).astype(np.uint8)
    if as_Image:
        img = Image.fromarray(img)
    return img

In [ ]:
class LineTensor(object):
    
    def __init__(
        self,
        linestring,
    ):
        self.init_ls = linestring
        self.pts = [p for p in self.init_ls.coords]
        self.tensor = torch.tensor(self.pts, requires_grad=True)
        self.init_loc_pt = self.ls.centroid
        self.init_loc_tensor = torch.tensor(np.array(self.init_loc_pt), requires_grad=True)
        
    @property
    def ls(self):
        return LineString(self.tensor.cpu().data.numpy())

# diffvg

In [ ]:
from copy import deepcopy
import fn

In [ ]:
class SpinLine(object):
    
    def __init__(
        self,
        offset_xy=None,
        angular_loc_deg=0.,
        radial_loc=0.,
        rotation_deg=0.,
        length=1.
    ):
        if offset_xy is None:
            offset_xy = np.array((0., 0.))
        self.offset_xy = offset_xy
        self.theta = angular_loc_deg
        self.r = radial_loc
        self.deg = rotation_deg
        self.length = length
        self.x = np.cos(self.theta) * self.r
        self.y = np.sin(self.theta) * self.r
        self.loc = np.array((self.x, self.y)) + self.offset_xy
        self.rel_coords = [np.array((np.cos(self.deg), np.sin(self.deg))) * self.length/2 * ii for ii in [-1, 1]]
        self.coords = [c + self.loc for c in self.rel_coords]
        self.line = LineString(self.coords)
         

In [ ]:
# Use GPU if available
dg.set_use_gpu(torch.cuda.is_available())

In [ ]:
width = 1200
height = 1600
drawbox = box(0, 0, width, height)
db = gp.Shape(drawbox)

In [ ]:
drawbox = box(0, 0, width, height)
db = gp.Shape(drawbox)

In [ ]:
out_shape = np.array((db.height, db.width)).round().astype(int)

In [ ]:
nft_id = fn.new_nft_id()

raster_videos_dir = Path('/home/naka/art/raster_videos')
nft_dir = raster_videos_dir.joinpath(nft_id)

if not nft_dir.exists():
    os.mkdir(nft_dir) 
    

# single

In [ ]:
t=0.00
center = np.array(db.p.centroid.xy).ravel()
n_circles = 7
max_rad = (db.width/2) * 0.8
min_rad = (db.width/2) * 0.05
radii = np.linspace(min_rad, max_rad, n_circles)
loc_xy_spacing = 55
sls = []

for radius in radii:
    circumference = radius * 360
    angular_locs = np.arange(0, circumference, loc_xy_spacing) / radius
    for angular_loc_deg in angular_locs:
        rotation_deg = angular_loc_deg * 4 + 60 * np.sin(0.01 * angular_loc_deg * t + 0.02*radius) + np.sin(0.00013 * angular_loc_deg) * 20
        length = 8. + np.sin(radius * angular_loc_deg) * 90 + np.sin(angular_loc_deg*0.001) * 40 + np.sin(0.00013 * angular_loc_deg) * 70
        rad = radius + np.sin(t * angular_loc_deg) * 20
        sl = SpinLine(offset_xy=center, angular_loc_deg=angular_loc_deg, radial_loc=rad, rotation_deg=rotation_deg, length=length)
        sls.append(LineString(sl.coords))

gp.merge_LineStrings(sls)

lts = [LineTensor(ls) for ls in sls]

canvas_width, canvas_height = width, height
num_control_points = torch.tensor([0])
shapes = []
shape_groups = []
for ii, lt in enumerate(lts):
    path = dg.Path(num_control_points = num_control_points,
                         points = lt.tensor,
                         is_closed = False,
                         stroke_width = torch.tensor(0.1))
    shapes.append(path)
    path_group = dg.ShapeGroup(shape_ids = torch.tensor([ii]),
                                     fill_color = torch.tensor([0.0, 0.0, 0.0, 0.0]),
                                     stroke_color = torch.tensor([1., 1., 1., 1]))
    shape_groups.append(path_group)


scene_args = dg.RenderFunction.serialize_scene(\
    canvas_width, canvas_height, shapes, shape_groups)
render = dg.RenderFunction.apply
img = render(canvas_width, # width
             canvas_height, # height
             2,   # num_samples_x
             2,   # num_samples_y
             0,   # seed
             None, # background_image
             *scene_args)
# target = img.clone()

angle_targets = [torch.tensor(0) for shape in shapes]

# init
rendered_img = finalize_image(img.cpu(), as_Image=True)

img = render(canvas_width, # width
             canvas_height, # height
             2,   # num_samples_x
             2,   # num_samples_y
             0,   # seed
             None, # background_image
             *scene_args)
img = finalize_image(img.cpu(), as_Image=True)


background = Image.new('RGBA', img.size, (0, 0, 0))

alpha_composite = Image.alpha_composite(background, img)
alpha_composite

In [ ]:
quality_val = 100
now = fn.get_time()
filepath = nft_dir.joinpath(f'{nft_id}_{now}_0000.jpeg')
alpha_composite.convert('RGB').save(filepath, quality=quality_val)

# movie

In [ ]:

ts = np.arange(0., 1., 0.001)
quality_val = 100

In [ ]:
for img_no, t in enumerate(tqdm(ts)):
    center = np.array(db.p.centroid.xy).ravel()
    n_circles = 10
    max_rad = (db.width/2) * 0.8
    min_rad = (db.width/2) * 0.05
    radii = np.linspace(min_rad, max_rad, n_circles)
    loc_xy_spacing = 85
    sls = []

    for radius in radii:
        circumference = radius * 360
        angular_locs = np.arange(0, circumference, loc_xy_spacing) / radius
        for angular_loc_deg in angular_locs:
            rotation_deg = angular_loc_deg * 0.5 + 40 * np.sin(t* 0.001 * radius ) + 40 * np.sin(0.01 * angular_loc_deg * t)
            length = 88. + np.sin(t)
            sl = SpinLine(offset_xy=center, angular_loc_deg=angular_loc_deg, radial_loc=radius, rotation_deg=rotation_deg, length=length)
            sls.append(LineString(sl.coords))

    gp.merge_LineStrings(sls)

    lts = [LineTensor(ls) for ls in sls]

    canvas_width, canvas_height = width, height
    num_control_points = torch.tensor([0])
    shapes = []
    shape_groups = []
    for ii, lt in enumerate(lts):
        path = dg.Path(num_control_points = num_control_points,
                             points = lt.tensor,
                             is_closed = False,
                             stroke_width = torch.tensor(0.1))
        shapes.append(path)
        path_group = dg.ShapeGroup(shape_ids = torch.tensor([ii]),
                                         fill_color = torch.tensor([0.0, 0.0, 0.0, 0.0]),
                                         stroke_color = torch.tensor([1., 1., 1., 1]))
        shape_groups.append(path_group)


    scene_args = dg.RenderFunction.serialize_scene(\
        canvas_width, canvas_height, shapes, shape_groups)
    render = dg.RenderFunction.apply
    img = render(canvas_width, # width
                 canvas_height, # height
                 2,   # num_samples_x
                 2,   # num_samples_y
                 0,   # seed
                 None, # background_image
                 *scene_args)
    # target = img.clone()

    angle_targets = [torch.tensor(0) for shape in shapes]

    # init
    rendered_img = finalize_image(img.cpu(), as_Image=True)

    img = render(canvas_width, # width
                 canvas_height, # height
                 2,   # num_samples_x
                 2,   # num_samples_y
                 0,   # seed
                 None, # background_image
                 *scene_args)
    img = finalize_image(img.cpu(), as_Image=True)


    background = Image.new('RGBA', img.size, (0, 0, 0))

    alpha_composite = Image.alpha_composite(background, img)
    filepath = nft_dir.joinpath(f'{nft_id}_{img_no:0004}.jpeg')
    alpha_composite.convert('RGB').save(filepath, quality=quality_val)

In [ ]:
filenames = [fp.as_posix() for fp in nft_dir.glob('.jpeg)')]

In [ ]:
import moviepy

In [ ]:
clipout = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(filenames, fps=10)
clipout.write_videofile(nft_dir.joinpathnpath(f'{nft_id}.mp4'))

# moire

In [ ]:
from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper

from scipy import spatial, stats
from scipy.ndimage import gaussian_filter
from scipy.integrate import odeint
# make page
paper_size = '11x14 inches'
border:float=30
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
center = drawbox.centroid

In [ ]:
n_lines = 8421
thetas = np.linspace(0, np.pi*27, n_lines)
radii = np.linspace(0.8, 28, n_lines)

In [ ]:
pts = []
for theta, radius in zip(thetas, radii):
    x = np.cos(theta) * radius - 0
    y = np.sin(theta) * radius + 0.
    pts.append(Point(x, y))

In [ ]:
def ode(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(b * u) + v * c
    dudt = np.cos(a * v * u) + u  * d
    dydt = [dvdt, dudt]
    return dydt

In [ ]:


t_max = 5.7
t = np.linspace(0, t_max, 41)
a = 0.1
b = 0.95
c = - 0.02
d = -0.02

all_polys = Polygon()

break_dist = 0.01

lines = []
lfs = MultiLineString()
allowed_counter = 0
for ii, pt in enumerate(tqdm(pts)):
    sol = odeint(ode, [pt.x, pt.y], t, args=(a, b, c, d))
    mpt = MultiPoint(sol)
    if ii == 0:
        ls = LineString(mpt)
        lfs = gp.merge_LineStrings([lfs, ls])
        lines.append(ls)
    else:
        allowed_counter = 0
        for _pt in mpt:
            dist = _pt.distance(lfs)
            # if dist < break_dist:
            #     break
            allowed_counter += 1
    if allowed_counter > 1:
        ls = LineString(mpt[:allowed_counter])
        lfs = gp.merge_LineStrings([lfs, ls])
        lines.append(ls)
        
lbs0 = gp.merge_LineStrings([l for l in lines if l.length > 0.9])     
lbs0 = gp.make_like(gp.merge_LineStrings(lbs0), drawbox)



In [ ]:
# Use GPU if available
dg.set_use_gpu(torch.cuda.is_available())

In [ ]:
width = 2000
height = 2000

In [ ]:
drawbox = box(0, 0, width, height)
db = gp.Shape(drawbox)

In [ ]:
sls = gp.make_like(gp.merge_LineStrings(lbs0), drawbox)

In [ ]:
out_shape = np.array((db.height, db.width)).round().astype(int)

In [ ]:
nft_id = fn.new_nft_id()

raster_videos_dir = Path('/home/naka/art/raster_videos')
nft_dir = raster_videos_dir.joinpath(nft_id)

if not nft_dir.exists():
    os.mkdir(nft_dir) 
    

In [ ]:
sls

In [ ]:
lts = []
for ls in sls:
    for ii in range(len(ls.coords)-1):
        sub_ls = LineString(ls.coords[ii:ii+2])
        lt = LineTensor(sub_ls)
        lts.append(lt)

In [ ]:


canvas_width, canvas_height = width, height
num_control_points = torch.tensor([0])
shapes = []
shape_groups = []
for ii, lt in enumerate(lts):
    path = dg.Path(num_control_points = num_control_points,
                         points = lt.tensor,
                         is_closed = False,
                         stroke_width = torch.tensor(0.45))
    shapes.append(path)
    path_group = dg.ShapeGroup(shape_ids = torch.tensor([ii]),
                                     fill_color = torch.tensor([0.0, 0.0, 0.0, 0.0]),
                                     stroke_color = torch.tensor([1., 1., 1., 0.6]))
    shape_groups.append(path_group)


scene_args = dg.RenderFunction.serialize_scene(\
    canvas_width, canvas_height, shapes, shape_groups)
render = dg.RenderFunction.apply
img = render(canvas_width, # width
             canvas_height, # height
             2,   # num_samples_x
             2,   # num_samples_y
             0,   # seed
             None, # background_image
             *scene_args)
# target = img.clone()

angle_targets = [torch.tensor(0) for shape in shapes]

# init
rendered_img = finalize_image(img.cpu(), as_Image=True)

img = render(canvas_width, # width
             canvas_height, # height
             2,   # num_samples_x
             2,   # num_samples_y
             0,   # seed
             None, # background_image
             *scene_args)
img = finalize_image(img.cpu(), as_Image=True)


background = Image.new('RGBA', img.size, (0, 0, 0))

alpha_composite = Image.alpha_composite(background, img)
alpha_composite

In [ ]:
quality_val = 100
now = fn.get_time()
filepath = nft_dir.joinpath(f'{nft_id}_{now}_0000.jpeg')
alpha_composite.convert('RGB').save(filepath, quality=quality_val)

In [ ]:
from genpen import subdivide as sd
from functools import partial
from genpen.grower import Grower, GrowerParams

In [ ]:
# Use GPU if available
dg.set_use_gpu(torch.cuda.is_available())

In [ ]:
drawbox

In [ ]:
# make page
paper_size = 'A2'
border:float=45
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

split_func = functools.partial(sd.split_random_bezier, x0=0.2, x1=0.75, n_eval_points=50)

xgen = stats.uniform(loc=0.4, scale=0.01).rvs
split_func = functools.partial(sd.split_along_longest_side_of_min_rectangle, xgen=xgen)

# x0gen = ss.uniform(loc=0.15, scale=0.01).rvs
# x1gen = ss.uniform(loc=0.65, scale=0.01).rvs
# split_func = functools.partial(sd.split_random_line_gen, x0gen=x0gen, x1gen=x1gen)

target = Point(140, 325)
target = drawbox.centroid

dist_from_center = partial(sd.distance_from_pt, target=target, p_range=(0.99, 0.3,), d_range=(0, 200))
cp = sd.ContinuePolicy(dist_from_center)
polys = sd.very_flex_rule_recursive_split(poly=drawbox, split_func=split_func, continue_func=cp, depth_limit=14, buffer_kwargs={'distance':1e-6})


bps = gp.merge_Polygons(polys)


   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

layers = []


for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = stats.norm(loc=0, scale=np.random.uniform(0.1, 1)).rvs
        bhf = gp.BezierHatchFill(
            spacing=np.random.uniform(0.1, 0.5),
            degrees=np.random.uniform(10,80),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=5,
            n_eval_points=6,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

for tolerance in [0.1, 0.3, 0.5, 0.7]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
width = 1200
height = 1600
drawbox = box(0, 0, width, height)
db = gp.Shape(drawbox)

In [ ]:
sls = gp.make_like(gp.merge_LineStrings(layer), drawbox)

In [ ]:
lts = []
for ls in sls:
    for ii in range(len(ls.coords)-1):
        sub_ls = LineString(ls.coords[ii:ii+2])
        lt = LineTensor(sub_ls)
        lts.append(lt)

In [ ]:


canvas_width, canvas_height = width, height
num_control_points = torch.tensor([0])
shapes = []
shape_groups = []
for ii, lt in enumerate(lts):
    path = dg.Path(num_control_points = num_control_points,
                         points = lt.tensor,
                         is_closed = False,
                         stroke_width = torch.tensor(0.1))
    shapes.append(path)
    path_group = dg.ShapeGroup(shape_ids = torch.tensor([ii]),
                                     fill_color = torch.tensor([0.0, 0.0, 0.0, 0.0]),
                                     stroke_color = torch.tensor([1., 1., 1., 1]))
    shape_groups.append(path_group)


scene_args = dg.RenderFunction.serialize_scene(\
    canvas_width, canvas_height, shapes, shape_groups)
render = dg.RenderFunction.apply
img = render(canvas_width, # width
             canvas_height, # height
             2,   # num_samples_x
             2,   # num_samples_y
             0,   # seed
             None, # background_image
             *scene_args)
# target = img.clone()

angle_targets = [torch.tensor(0) for shape in shapes]

# init
rendered_img = finalize_image(img.cpu(), as_Image=True)

img = render(canvas_width, # width
             canvas_height, # height
             2,   # num_samples_x
             2,   # num_samples_y
             0,   # seed
             None, # background_image
             *scene_args)
img = finalize_image(img.cpu(), as_Image=True)


background = Image.new('RGBA', img.size, (0, 0, 0))

alpha_composite = Image.alpha_composite(background, img)
alpha_composite

In [ ]:
nft_id = fn.new_nft_id()

raster_videos_dir = Path('/home/naka/art/raster_videos')
nft_dir = raster_videos_dir.joinpath(nft_id)

if not nft_dir.exists():
    os.mkdir(nft_dir) 
    

In [ ]:
quality_val = 100
now = fn.get_time()
filepath = nft_dir.joinpath(f'{nft_id}_{now}_0000.jpeg')
alpha_composite.convert('RGB').save(filepath, quality=quality_val)

# simple

In [ ]:

diffvg_images_dir = Path('/home/naka/art/diffvg_images')


In [ ]:
width = 1600
height = 1600
drawbox = box(0, 0, width, height)
db = gp.Shape(drawbox)

In [ ]:
paper.page_format_mm

In [ ]:
sk = vsketch.Vsketch()
sk.size(f'{width}mmx{height}mm')
sk.scale('1mm')
sk.penWidth('0.3mm')


In [ ]:
n_circles = 1

In [ ]:
circles = [db.p.centroid.buffer(600) for ii in range(n_circles)]
    

In [ ]:
n_eval_points = 50

In [ ]:
clipped_filled_polys = []
for c in circles:
    filled = gp.BezierHatchFill(
        poly_to_fill=c,
        spacing=20,
        degrees=0,
        xjitter_func=stats.norm(loc=0, scale=0.1).rvs, 
        yjitter_func=stats.norm(loc=0, scale=5).rvs,
        fill_inscribe_buffer=1.4,
        n_nodes_per_line=10,
        n_eval_points=40,
        alternate_direction=False,
    )
    fills = filled.fill
    random_walk = gp.gaussian_random_walk(len(fills), step_init=0.5, step_mu=0., step_std=3, scale=True)
    clipped_lss = []
    for ii, ls in enumerate(fills):
        eval_pts = np.linspace(0, random_walk[ii], n_eval_points)
        clipped_ls = LineString([ls.interpolate(pt, normalized=True) for pt in eval_pts])
        clipped_lss.append(clipped_ls)
    clipped_filled_polys.append(gp.merge_LineStrings(clipped_lss))

In [ ]:
gp.merge_LineStrings(clipped_filled_polys)

In [ ]:
sls = gp.merge_LineStrings(clipped_filled_polys)

In [ ]:
lts = []
for ls in sls:
    for ii in range(len(ls.coords)-1):
        sub_ls = LineString(ls.coords[ii:ii+2])
        lt = LineTensor(sub_ls)
        lts.append(lt)

In [ ]:


canvas_width, canvas_height = width, height
num_control_points = torch.tensor([0])
shapes = []
shape_groups = []
for ii, lt in enumerate(lts):
    path = dg.Path(num_control_points = num_control_points,
                         points = lt.tensor,
                         is_closed = False,
                         stroke_width = torch.tensor(0.1))
    shapes.append(path)
    path_group = dg.ShapeGroup(shape_ids = torch.tensor([ii]),
                                     fill_color = torch.tensor([0.0, 0.0, 0.0, 0.0]),
                                     stroke_color = torch.tensor([1., 1., 1., 0.8]))
    shape_groups.append(path_group)


scene_args = dg.RenderFunction.serialize_scene(\
    canvas_width, canvas_height, shapes, shape_groups)
render = dg.RenderFunction.apply
img = render(canvas_width, # width
             canvas_height, # height
             4,   # num_samples_x
             4,   # num_samples_y
             0,   # seed
             None, # background_image
             *scene_args)
img = finalize_image(img.cpu(), as_Image=True)


background = Image.new('RGBA', img.size, (0, 0, 0))

alpha_composite = Image.alpha_composite(background, img)
alpha_composite

In [ ]:


nft_id = fn.new_nft_id()
quality_val = 100
now = fn.get_time()
filepath = diffvg_images_dir.joinpath(f'{nft_id}.jpeg')
alpha_composite.convert('RGB').save(filepath, quality=quality_val)

# flow beam graph

In [ ]:
import networkx as nx
class GraphGram(object):
    
    def __init__(self, graph, xoff=0, yoff=0, scale=1, layout_method='kamada_kawai_layout'):
        self.graph = graph
        self._nodes = None
        self.xoff = xoff
        self.yoff = yoff
        self.scale = scale
        self.layout_method = layout_method
    
    @property
    def center(self):
        return np.array((self.xoff, self.yoff))
    
    @property
    def edges(self):
        return list(self.graph.edges)
    
    @property
    def layout_function(self):
        try:
            f = getattr(nx.layout, self.layout_method)
        except AttributeError:
            layout_functions = [a for a in dir(nx.layout) if 'layout' in a]
            error_string = f'''{self.layout_method} not found in networkx.layout module; 

choose from {layout_functions}
            '''
            print(error_string)
        return f
    
    @functools.lru_cache
    def get_layout(self, *args, **kwargs):
        self._nodes = self.layout_function(
            self.graph,
            scale=self.scale,
            center=self.center,
            *args, **kwargs)
    
    @property
    def nodes(self):
        if self._nodes is None:
            self.get_layout()
        return self._nodes
    
    @property
    def node_pts(self):
        return {k:Point(xy) for k, xy in self.nodes.items()}
    
    @property
    def pts(self):
        return MultiPoint([p for p in self.node_pts.values()])
    
    @property
    def lines(self):
        lines = []
        for n0,n1 in self.edges:
            p0 = self.node_pts[n0]
            p1 = self.node_pts[n1]
            lines.append(LineString([p0, p1]))
        return MultiLineString(lines)    
        

In [ ]:

diffvg_images_dir = Path('/home/naka/art/diffvg_images')


In [ ]:
width = 1600
height = 1600
drawbox = box(0, 0, width, height)
db = gp.Shape(drawbox)

In [ ]:
# make page

In [ ]:
DEGREE = 32
SCALE = 200
(xbins, ybins), (xs, ys) = gp.overlay_grid(drawbox, xstep=400, ystep=400, flatmesh=True)

In [ ]:
p_gen = lambda x: np.interp(x, [xs.min(), xs.max()], [0., 0.5] )
_p_gen = gp.make_callable(p_gen)

In [ ]:
k_gen = 2
_k_gen = gp.make_callable(k_gen)

In [ ]:
df = pd.DataFrame({
    'x':xs,
    'y':ys,
    'k':_k_gen(xs),
    'p':_p_gen(xs)
})
df['k'] = df['k'].astype(int)

In [ ]:
new_rows = []
for i, row in df.iterrows():
    k = row['k'].astype(int)
    G = nx.connected_watts_strogatz_graph(n=DEGREE, k=k, p=row['p'])
    gg = GraphGram(graph=G, layout_method='spring_layout',
                   xoff=row['x'], yoff=row['y'], scale=SCALE)
    
    bezs = []
    for ls in gg.lines:
        bez = gp.LineString_to_jittered_bezier(
            ls, xstd=0., ystd=0., normalized=True, n_eval_points=4)
        bezs.append(bez)
    bezs = gp.merge_LineStrings(bezs)
    new_row = row.to_dict()
    new_row['geometry'] = bezs
    new_rows.append(new_row)
    
gdf = geopandas.GeoDataFrame(new_rows)
layers = []
layers.append(gp.merge_LineStrings(gdf.geometry))

In [ ]:
layers[0]

In [ ]:
buffer_gen = stats.uniform(loc=18, scale=20).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-0.8, high=-1.)
angles_gen = stats.uniform(loc=0, scale=360).rvs
angles_gen = gp.make_callable(80)
d_translate_factor_gen = stats.uniform(loc=0.5, scale=0.8).rvs

In [ ]:
fills = []
all_polys = Polygon()
for i, row in gdf.iterrows():
    p = row.geometry.buffer(0.5, cap_style=2, join_style=2, resolution=8)
    p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.15, 0.15, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
        
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

In [ ]:
all_polys

In [ ]:
fills = [f for f in fills if f.length > 0]
all_fills = gp.merge_LineStrings(fills)
all_fills = gp.make_like(all_fills, db.p.buffer(-20))


In [ ]:
lts = []
for ls in all_fills:
    for ii in range(len(ls.coords)-1):
        sub_ls = LineString(ls.coords[ii:ii+2])
        lt = LineTensor(sub_ls)
        lts.append(lt)

In [ ]:


canvas_width, canvas_height = width, height
num_control_points = torch.tensor([0])
shapes = []
shape_groups = []
for ii, lt in enumerate(lts):
    path = dg.Path(num_control_points = num_control_points,
                         points = lt.tensor,
                         is_closed = False,
                         stroke_width = torch.tensor(0.2))
    shapes.append(path)
    path_group = dg.ShapeGroup(
        shape_ids = torch.tensor([ii]),
        fill_color = torch.tensor([0.0, 0.0, 0.0, 0.0]),
        stroke_color = torch.tensor([1., 1., 1., 0.8]))
    shape_groups.append(path_group)


scene_args = dg.RenderFunction.serialize_scene(\
    canvas_width, canvas_height, shapes, shape_groups)
render = dg.RenderFunction.apply
img = render(canvas_width, # width
             canvas_height, # height
             4,   # num_samples_x
             4,   # num_samples_y
             0,   # seed
             None, # background_image
             *scene_args)
img = finalize_image(img.cpu(), as_Image=True)


background = Image.new('RGBA', img.size, (0, 0, 0))

alpha_composite = Image.alpha_composite(background, img)
alpha_composite

In [ ]:

diffvg_images_dir = Path('/home/naka/art/diffvg_images')


In [ ]:


nft_id = fn.new_nft_id()
quality_val = 100
now = fn.get_time()
filepath = diffvg_images_dir.joinpath(f'{nft_id}.jpeg')
alpha_composite.convert('RGB').save(filepath, quality=quality_val)

# bez circles

In [ ]:
width = 1600
height = 1600
drawbox = box(0, 0, width, height)
db = gp.Shape(drawbox)

In [ ]:
bps = gp.circle_pack_within_poly(drawbox, rads=[400,200, 100, 55,35])

In [ ]:
bps

In [ ]:
bps2 = gp.circle_pack_within_poly(drawbox, rads=[400,200, 100, 55,35])

In [ ]:
bps = bps.difference(bps2.boundary.buffer(1.5))

In [ ]:
n_layers = 1

In [ ]:
layers = []
for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = stats.norm(loc=0, scale=np.random.uniform(3, 8.5)).rvs
        dist_from_center = p.centroid.distance(bps.centroid)
        a = np.interp(dist_from_center, [0, 800], [0, 720])
        bhf = gp.BezierHatchFill(
            spacing=np.random.uniform(0.8, 1.2),
            degrees=a,
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=40,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
layer

In [ ]:
mlayers = []
for layer in tqdm(layers):
    mlayers.append(layer.buffer(0.001).buffer(-0.001).boundary)

In [ ]:
mlayers = [gp.merge_LineStrings([l for l in layer if l.length > 0.2]) for layer in mlayers]

In [ ]:
sns.displot([np.log10(l.length) for l in mlayers[0]])

In [ ]:
lss = gp.merge_LineStrings(layers)

In [ ]:
lts = []
for ls in lss:
    for ii in range(len(ls.coords)-1):
        sub_ls = LineString(ls.coords[ii:ii+2])
        lt = LineTensor(sub_ls)
        lts.append(lt)

In [ ]:
len(lts)

In [ ]:


canvas_width, canvas_height = width, height
num_control_points = torch.tensor([0])
shapes = []
shape_groups = []
for ii, lt in enumerate(lts):
    path = dg.Path(num_control_points = num_control_points,
                         points = lt.tensor,
                         is_closed = False,
                         stroke_width = torch.tensor(0.1))
    shapes.append(path)
    path_group = dg.ShapeGroup(
        shape_ids = torch.tensor([ii]),
        fill_color = torch.tensor([0.0, 0.0, 0.0, 0.0]),
        stroke_color = torch.tensor([1., 1., 1., 0.8]))
    shape_groups.append(path_group)


scene_args = dg.RenderFunction.serialize_scene(\
    canvas_width, canvas_height, shapes, shape_groups)
render = dg.RenderFunction.apply
img = render(canvas_width, # width
             canvas_height, # height
             4,   # num_samples_x
             4,   # num_samples_y
             0,   # seed
             None, # background_image
             *scene_args)
img = finalize_image(img.cpu(), as_Image=True)


background = Image.new('RGBA', img.size, (0, 0, 0))

alpha_composite = Image.alpha_composite(background, img)
alpha_composite

In [ ]:

diffvg_images_dir = Path('/home/naka/art/diffvg_images')


In [ ]:


nft_id = fn.new_nft_id()
quality_val = 100
now = fn.get_time()
filepath = diffvg_images_dir.joinpath(f'{nft_id}.jpeg')
alpha_composite.convert('RGB').save(filepath, quality=quality_val)